# Definir diretório raiz

In [ ]:
!pip install imgaug
!pip install Cython
!pip install pycocotools
!pip install kaggle
!pip install pillow

In [ ]:
#!pip install python3.6-tk
#!pip install natsort
#!pip install pandas
#!pip install numpy
#!pip install Pillow

In [1]:
from tkinter import filedialog
from tkinter import *
import os 
import pandas as pd
from PIL import Image

root = Tk()
#root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
root.dir_raiz = filedialog.askdirectory()
print (root.dir_raiz)


os.chdir(root.dir_raiz)




/home/dblab/projects/Mask_RCNN_v3_dataAug


# DEFFINIÇÂO DE Nome dos Arquivos E Quantidades

In [2]:


#root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
root.filename = filedialog.askdirectory()
print (root.filename)

imagens_produto_path = root.filename + "/" 
nome_produto = root.filename.split("/")
nome_produto = nome_produto[len(nome_produto)-1]


root.destroy()


/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2


In [ ]:
#pip install natsort

# Analisar fotos do meu repositorio

In [4]:
from os import listdir
from os.path import isfile, join
from natsort import natsorted

arquivos_imagens = [f for f in natsorted(listdir(imagens_produto_path)) if isfile(join(imagens_produto_path, f))]

print(arquivos_imagens)


['garrafa2(0).png', 'garrafa2(1).png', 'garrafa2(2).png', 'garrafa2(3).png', 'garrafa2(4).png', 'garrafa2(5).png', 'garrafa2(6).png', 'garrafa2(7).png', 'garrafa2(8).png', 'garrafa2(9).png', 'garrafa2(10).png', 'garrafa2(11).png', 'garrafa2(12).png', 'garrafa2(13).png', 'garrafa2(14).png', 'garrafa2(15).png', 'garrafa2(16).png', 'garrafa2(17).png', 'garrafa2(18).png', 'garrafa2(19).png', 'garrafa2(20).png', 'garrafa2(21).png', 'garrafa2(22).png', 'garrafa2(23).png', 'garrafa2(24).png', 'garrafa2(25).png', 'garrafa2(26).png', 'garrafa2(27).png', 'garrafa2(28).png', 'garrafa2(29).png', 'garrafa2(30).png', 'garrafa2(31).png', 'garrafa2(32).png', 'garrafa2(33).png', 'garrafa2(34).png', 'garrafa2(35).png', 'garrafa2(36).png', 'garrafa2(37).png', 'garrafa2(38).png', 'garrafa2(39).png', 'garrafa2(40).png', 'garrafa2(41).png', 'garrafa2(42).png', 'garrafa2(43).png', 'garrafa2(44).png', 'garrafa2(45).png', 'garrafa2(46).png', 'garrafa2(47).png', 'garrafa2(48).png', 'garrafa2(49).png', 'garrafa2

# Prepare Model

In [12]:
import sys
import random
import math
import numpy as np

import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import pycocotools

# Root directory of the project
#ROOT_DIR = os.path.abspath("../")
#print(ROOT_DIR)
ROOT_DIR = root.dir_raiz
print(ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
#import coco
from samples.coco import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
#COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
#if not os.path.exists(COCO_MODEL_PATH):
utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = imagens_produto_path;

/home/dblab/projects/Mask_RCNN_v3_dataAug
... done downloading pretrained model!


# Create Inference Object

In [13]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

In [14]:


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

print(MODEL_DIR)
print(COCO_MODEL_PATH)
# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)



/home/dblab/projects/Mask_RCNN_v3_dataAug/logs
/home/dblab/projects/mask_rcnn_coco.h5


In [15]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Prediction and Visualization

# Cria diretorio com nome do produto na Pasta de OUTPUT das mascaras e coordenadas

In [16]:
path = "output/"+nome_produto

try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)

Successfully created the directory output/garrafa_2 


# Importando métodos para Data Augmentation 

In [17]:
import imageio
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
%matplotlib inline
ia.seed(1)

#image = imageio.imread(filename)
#image = ia.imresize_single_image(image, (298, 447))

#xmin = 127
#xmax = 237
#ymin = 104
#ymax = 437


#ia.imshow(bbs.draw_on_image(image, size=2))

# Passa as imagens de produto da pasta para gerar coordenas e mascaras

In [18]:


coordenadas = np.empty([1, 5], dtype=int)

df_fileName = pd.DataFrame()
nomes_masks = pd.DataFrame()

#print(coordenadas.shape)

for i in range(0,len(arquivos_imagens)-1):
    #print(len(arquivos_imagens))

#for i in range(0, 1):
    
    path = imagens_produto_path + arquivos_imagens[i]
    image = skimage.io.imread(path)
    
    print(path)
    
    df_fileName.loc[len(df_fileName),0] = "images/"+nome_produto+"/"+arquivos_imagens[i]
    # Run detection
    results = model.detect([image], verbose=0)
  
    # Visualize results
    r = results[0]

    b = np.empty([1, 5], dtype=int)
    #print('b 0 ',b)

    b[0,:-1] = r['rois'][0]
  
    

    b[0,4] = i

    coordenadas = np.append(coordenadas, b, axis=0)
  
    imagem_export = r['masks'][:,:,0]
    im = Image.fromarray(imagem_export.astype('uint8')*255)
    
    im.save("output/"+nome_produto+"/mask_("+"{}".format(i)+").png")
    nomes_masks.loc[len(nomes_masks),0] = "output/"+nome_produto+"/mask_("+"{}".format(i)+").png"
    
    bbs = BoundingBoxesOnImage([
    BoundingBox(x1=b[0,1], x2=b[0,3], y1=b[0,0], y2=b[0,2]),
    #BoundingBox(x1=0.4*xmin, x2=0.65*xmax, y1=0.1*ymin, y2=0.4*ymax)
    ], shape=image.shape)
    
    posx = round(random.uniform(-0.2,0.2), 2)
    posy = round(random.uniform(-0.2,0.2), 2)
    scl = round(random.uniform(0.2,1.0), 2)
    
    seq = iaa.Sequential([
    iaa.GammaContrast(1.5),
    iaa.Affine(translate_percent={"x": posx, "y": posy}, scale=scl)
    ])
    #print(type(image))
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
    
    co_au = np.empty([1, 5], dtype=float)
    co_au[0,:] = [round(bbs_aug.bounding_boxes[0].y1,1),round(bbs_aug.bounding_boxes[0].x1,1),round(bbs_aug.bounding_boxes[0].y2,1),round(bbs_aug.bounding_boxes[0].x2,1), i]
    
    #print(type(co_au))
    #print(type(im))
    im2 = Image.fromarray(image_aug)
    im2.save("output/"+nome_produto+"/aum_("+"{}".format(i)+").png")
    df_fileName.loc[len(df_fileName),0] = "output/"+nome_produto+"/aum_("+"{}".format(i)+").png"
    
    coordenadas = np.append(coordenadas, co_au, axis=0)
    
    image_m = skimage.io.imread("output/"+nome_produto+"/mask_("+"{}".format(i)+").png")
    image_aug_2, bbs_aug_2 = seq(image=image_m, bounding_boxes=bbs)
    im_mask_aug = Image.fromarray(image_aug_2)
    im_mask_aug.save("output/"+nome_produto+"/aum_mask_("+"{}".format(i)+").png")
    nomes_masks.loc[len(nomes_masks),0] = "output/"+nome_produto+"/aum_mask_("+"{}".format(i)+").png"
    
    
coordenadas = np.delete(coordenadas, 0, 0)

names = ["Ymin", "Xmin", "Ymax", "Xmax", "id"]

df = pd.DataFrame(coordenadas, columns=names)



df["name"] = df_fileName
df["mask"] = nomes_masks

#print(df_fileName.shape)

df.to_csv("output/"+nome_produto+"/"+nome_produto+".csv", index=True, header=True, sep=',')

#print(df)
print("done")


/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(0).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(1).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(2).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(3).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(4).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(5).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(6).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(7).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(8).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(9).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(10).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(11).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(12).png
/home/dblab/projects/M

/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(108).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(109).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(110).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(111).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(112).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(113).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(114).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(115).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(116).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(117).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(118).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(119).png
/home/dblab/projects/Mask_RCNN_v3_dataAug/images/garrafa_2/garrafa2(120).png

# Background Removal

In [ ]:


# Load a random image from the images folder
#for i in range(0,351):
for i in range(0,1):
    path = 'images/garrafa_2/garrafa2('+"{}".format(i)+').png'
    image = skimage.io.imread(path)
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
    results = model.detect([image], verbose=1)

In [ ]:
# Visualize results
r = results[0]

r['masks'].shape

#r['masks'].transpose(1, 0, 2).shape

image.shape

#print(r['masks'])

#np.savetxt("images/coord_mascara.csv", r['masks'], delimiter=",", fmt='%s')
#print(done)

#visualize.apply_mask(image[:,:,:2], r['masks'], [1, 0, 0], alpha=1.0)


visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                       class_names, r['scores'])

#visualize.save_image(image, "mascara_197", r['rois'], r['masks'], r['class_ids'], 
#                        r['scores'], class_names, mode = 3)

In [ ]:
#PARA TODAS AS IMAGENS GERAR MASCARAS

# Load a random image from the images folder
for i in range(0,n_aquivos):

    path = 'images/garrafa_2/garrafa2('+"{}".format(i)+').png'
    image = skimage.io.imread(path)
#image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))

# Run detection
    results = model.detect([image], verbose=1)
    
    # Visualize results
    r = results[0]
    
  
    if results[0]['rois'].shape == (1,4):
    
        visualize.save_image(image, 'mask_('+"{}".format(i)+')', r['rois'], r['masks'], r['class_ids'], 
                        r['scores'], class_names, mode = 3)
        print(i)
     
print('done');
    
    